In [ ]:
pip install -U weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.8/583.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.2 MB/s eta 0:00:00


# Sparse Dense and Hybrid Search

## Remove old Weaviate DB files

In [ ]:
!rm -rf ~/.local/share/weaviate

## Recreate the example
With the same data as in the previous lesson

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))

def json_print(data):
    print(json.dumps(data, indent=2))

<class 'list'> 10


In [ ]:
# import weaviate, os
# from weaviate import EmbeddedOptions
# import openai

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

# client = weaviate.Client(
#     embedded_options=EmbeddedOptions(),
#     additional_headers={
#         "X-OpenAI-Api-BaseURL": os.environ['OPENAI_API_BASE'],
#         "X-OpenAI-Api-Key": openai.api_key,  # Replace this with your actual key
#     }
# )
# print(f"Client created? {client.is_ready()}")

In [ ]:
import os

import weaviate
from dotenv import load_dotenv, find_dotenv

# v4 config helpers
from weaviate.classes.config import Property, DataType, Configure
from weaviate.classes.query import MetadataQuery  # (optional, for scores if you want)



# --- 1) Setup & data ---
_ = load_dotenv(find_dotenv())
# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]              # required
# OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE", None)  # optional (proxy / Azure-style endpoint)
OPENAI_API_BASE = 'http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate'

In [ ]:

def show(title, response):
    print(f"\n=== {title} ===")
    for i, obj in enumerate(response.objects, 1):
        q = obj.properties.get("question")
        a = obj.properties.get("answer")
        print(f"{i}. Q: {q}\n   A: {a}\n")


In [ ]:
# --- 2) Connect to Embedded Weaviate (v4 style) ---
# Tip: Pin the version for reproducibility. You can bump to a newer server version later if needed.
client = weaviate.connect_to_embedded(
    # version="1.30.0",
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        # If you use a custom OpenAI-compatible endpoint/proxy, uncomment the next line.
        "X-OpenAI-BaseURL": OPENAI_API_BASE  # header spelling that Weaviate recognizes
    },
    # environment_variables={"LOG_LEVEL": "error"}  # optional: quieter logs
)
print("Client connected.")


INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.30.0/weaviate-v1.30.0-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 1459


Client connected.


In [ ]:
# # Uncomment the following two lines if you want to run this block for a second time.
# if client.schema.exists("Question"):
#    client.schema.delete_class("Question")

# class_obj = {
#     "class": "Question",
#     "vectorizer": "text2vec-openai",  # Use OpenAI as the vectorizer
#     "moduleConfig": {
#         "text2vec-openai": {
#             "model": "ada",
#             "modelVersion": "002",
#             "type": "text",
#             "baseURL": os.environ["OPENAI_API_BASE"]
#         }
#     }
# }

# client.schema.create_class(class_obj)

In [ ]:
# --- 3) (Re)create the collection in v4 ---
COLLECTION = "Question"


# Clean up if you re-run the notebook
try:
    if client.collections.exists(COLLECTION):
        client.collections.delete(COLLECTION)
except Exception as e:
    # If your client lacks permissions, .exists() may return False; ignore and continue
    pass


In [ ]:
# Create the collection with OpenAI vectorizer
# Use the latest text-embedding-3 model family (small is fast & good enough for the tutorial).
# If you route via a proxy / Azure-like base URL, pass base_url=OPENAI_API_BASE.
vector_cfg = Configure.Vectors.text2vec_openai(
    model="text-embedding-3-small",
    # dimensions can be omitted for defaults; for small family it's typically 1536
    # dimensions=1536,
    base_url=OPENAI_API_BASE if OPENAI_API_BASE else None
)

questions = client.collections.create(
    name=COLLECTION,
    properties=[
        Property(name="answer",   data_type=DataType.TEXT),
        Property(name="question", data_type=DataType.TEXT),
        Property(name="category", data_type=DataType.TEXT),
    ],
    vector_config=vector_cfg
)
print("Collection created.")


Collection created.


In [ ]:
# with client.batch.configure(batch_size=5) as batch:
#     for i, d in enumerate(data):  # Batch import data

#         print(f"importing question: {i+1}")

#         properties = {
#             "answer": d["Answer"],
#             "question": d["Question"],
#             "category": d["Category"],
#         }

#         batch.add_data_object(
#             data_object=properties,
#             class_name="Question"
#         )

In [ ]:

# --- 4) Batch import (v4 batching) ---
col = client.collections.get(COLLECTION)

# Fixed-size batching similar to v3's batch_size
with col.batch.fixed_size(batch_size=5) as batch:
    for i, d in enumerate(data):
        print(f"Importing question {i+1}/{len(data)}")
        batch.add_object(
            properties={
                "answer": d["Answer"],
                "question": d["Question"],
                "category": d["Category"],
            }
            # If you already have vectors, you can provide `vector=...` here.
        )


Importing question 1/10
Importing question 2/10
Importing question 3/10
Importing question 4/10
Importing question 5/10
Importing question 6/10
Importing question 7/10
Importing question 8/10
Importing question 9/10
Importing question 10/10


ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 5', 'error': 'WeaviateInsertManyAllFailedError(\'Every object failed during insertion. Here is the set of all errors: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai on 127.0.0.11:53: no such host\')'}
ERROR:weaviate-client:{'message': 'Failed to send 5 objects in a batch of 5. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 5', 'error': 'WeaviateInsertManyAllFailedError(\'Every object failed during insertion. Here is the set of all errors: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai on 127.0.0.11:53: no such host\')'}
ERROR:weaviate-client:{'message': 'Failed to send 5 obj

## Queries

### Dense Search

In [ ]:
# response = (
#     client.query
#     .get("Question", ["question", "answer"])
#     .with_near_text({"concepts":["animal"]})
#     .with_limit(3)
#     .do()
# )

# json_print(response)

In [ ]:

# --- 5) Queries: near_text, BM25, and hybrid (v4) ---
# near_text (vector search)
resp = col.query.near_text(
    query="animal",
    limit=3,
    return_properties=["question", "answer"]
    # return_metadata=MetadataQuery(distance=True)  # Uncomment to also see vector distance
)
show("near_text(query='animal')", resp)


### Sparse Search - BM25

In [ ]:
# response = (
#     client.query
#     .get("Question",["question","answer"])
#     .with_bm25(query="animal")
#     .with_limit(3)
#     .do()
# )

# json_print(response)

In [ ]:

# BM25 (keyword search)
resp = col.query.bm25(
    query="animal",
    limit=3,
    return_properties=["question", "answer"]
    # properties=["question", "answer"]  # optionally restrict searched props
)
show("bm25(query='animal')", resp)


### Hybrid Search

In [ ]:
# response = (
#     client.query
#     .get("Question",["question","answer"])
#     .with_hybrid(query="animal", alpha=0.5)
#     .with_limit(3)
#     .do()
# )

# json_print(response)

In [ ]:
# response = (
#     client.query
#     .get("Question",["question","answer"])
#     .with_hybrid(query="animal", alpha=0)
#     .with_limit(3)
#     .do()
# )

# json_print(response)

In [ ]:
# response = (
#     client.query
#     .get("Question",["question","answer"])
#     .with_hybrid(query="animal", alpha=1)
#     .with_limit(3)
#     .do()
# )

# json_print(response)

In [ ]:

# Hybrid (BM25 + vector) with different alpha values
for alpha in (0.5, 0.0, 1.0):
    resp = col.query.hybrid(
        query="animal",
        alpha=alpha,            # 0.0 -> pure BM25/sparse search.    1.0 -> pure vector/dense search
        limit=3,
        return_properties=["question", "answer"]
    )
    show(f"hybrid(query='animal', alpha={alpha})", resp)


In [ ]:

# --- 6) Cleanup (optional) ---
# client.collections.delete(COLLECTION)

# Close the client when you’re done
client.close()
print("Done.")


Done.
